In [299]:
import json
with open('testSet/ML100kEvaluationRecAlpaca.json', 'r') as file:
    data = json.load(file)

In [300]:
len(data)

200

In [301]:
import ast
gt = []
for i in range(200):
    gt.append(data[i]['testGT'])

In [302]:
pred = []
for i in range(200):
    pred.append(data[i]['output2'])

In [303]:
def Hit(A, test_set):
    dcg = 0
    r_i = 0
    for i in range(len(A)):
        
        if A[i] in test_set:
            r_i = 1
            return r_i
    return r_i

In [304]:
import numpy as np
import numpy as np
from sklearn.metrics import ndcg_score

def compute_precision_recall(targets, predictions, k):  
    num_hit = [pred for pred in predictions if any(pred[:int(1*len(pred))] == tar[:int(1*len(tar))] for tar in targets)]
    num_hit = len(num_hit)
    precision = float(num_hit) / len(predictions)    
    recall = float(num_hit) / len(targets)    
    return precision, recall


def NDCG(targets, predicitions):
    test_movies = list(set(movie for sublist in predicitions for movie in sublist)) # unique items
    y_score = np.zeros((len(predicitions), len(test_movies))) # convert to score
    for i, rec_list in enumerate(predicitions):
        for j, item in enumerate(test_movies):
            if item in rec_list:
                y_score[i, j] = len(test_movies) - rec_list.index(item)# get a list of scores, match and early item have higher score

    # Convert test_sets to y_true
    y_true = np.zeros((len(targets), len(test_movies)))
    for i, test_list in enumerate(targets):
        for j, item in enumerate(test_movies):
            if item in test_list:
                y_true[i, j] = 1 # if hit 1 if not 0
    return ndcg_score(y_true, y_score, k = 5)

In [305]:
import re
def texta(text):
    recommended_movies = []
    for line in text.split('\n'):
        if line.startswith(("1. ", "2. ", "3. ", "4. ", "5. ")):
            movie = re.search(r'\"(.*?)\"', line)
            if movie:
                recommended_movies.append(movie.group(1))
    return recommended_movies

In [306]:
import re
pt_movies = []
for a in range(200):
    text = pred[a]
    pattern =  r'\d+\. (.+?) -'
    recommended_movies = re.findall(pattern, text)
    if len(list(set(recommended_movies))) != 5:
        recommended_movies = re.findall(r'\d+\.\s(.*?)\s\(\d+\)', text)
        if len(list(set(recommended_movies))) != 5:
            recommended_movies = re.findall(r'\d+\.\s(.*?)\s\(Liked\)', text)
            if len(list(set(recommended_movies))) != 5:
                recommended_movies = re.findall(r'\"(.*?)\"', text)
                if len(list(set(recommended_movies))) != 5:
                    recommended_movies = texta(text)
                    if len(recommended_movies) != 5:
                        recommended_movies = re.findall(r'\"(.*?)\"', text)
                        if len(recommended_movies) != 5:
                            recommended_movies = re.findall(r'^\d\.\s(.*?)\s-', text, re.MULTILINE)
                            recommended_movies = recommended_movies[:5]
                            if len(recommended_movies) != 5:
                                recommended_movies = re.findall(r'\d\.\s(.*?)\n', text)
                                if len(recommended_movies) != 5:
                                    recommended_movies = re.findall(r'\d\.\s(.*?)(?:\n|$)', text)
                                    recommended_movies[-1] = recommended_movies[-1].replace('</s>', '')
                                    if len(recommended_movies) != 5:
                                        print(a)
    titles = [title.replace('"', '') for title in recommended_movies]
    pt_movies.append(list(set(titles)))  

160
166
172


In [307]:
m = [] 
for element in pt_movies:
    movies_no_year = [re.sub(r' \(\d{4}\)', '', movie) for movie in element]
    movies_no_score = [re.sub(r' \(\d\.\d\/10\)', '', movie) for movie in movies_no_year]
    movies_no_director = [re.sub(r' \([^)]*\)', '', movie) for movie in movies_no_score]
    movie_names = [desc.split(":")[0] for desc in movies_no_director]
    m.append(movie_names)

In [308]:
pre = 0.0
recall = 0.0
for i in range(200):
    titles = m[i]
    if titles == []:
        continue
    temp_pre, temp_recall = compute_precision_recall(gt[i],titles,1)
    pre = pre + temp_pre
    recall = recall + temp_recall
    
print('The NDCG@5 loss is computed to be: {}'.format(NDCG(gt,m)))
print('The pre@5 loss is computed to be: {}'.format(pre /200))
print('The recall@5 loss is computed to be: {}'.format(recall/200))

The NDCG@5 loss is computed to be: 0.3708149254300115
The pre@5 loss is computed to be: 0.33366666666666717
The recall@5 loss is computed to be: 0.1770160442286101
